In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SQL Tables').getOrCreate()

25/06/10 02:41:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Managed Table

In [43]:
df = spark.read.format('csv').option('inferScema', True).option('header', True).load('/data/customers1.csv')

In [44]:
df2 = df.withColumnRenamed("2023-10-07", "date")

In [45]:
df2.show()

+-----+--------------+---------+
|India|          date|    False|
+-----+--------------+---------+
|17455|Customer_17455|    Delhi|
|17456|Customer_17456|Bangalore|
|17457|Customer_17457|  Kolkata|
|17458|Customer_17458|Hyderabad|
|17459|Customer_17459|Bangalore|
|17460|Customer_17460|     Pune|
|17461|Customer_17461|  Chennai|
|17462|Customer_17462|Bangalore|
|17463|Customer_17463|   Mumbai|
|17464|Customer_17464|  Kolkata|
|17465|Customer_17465|   Mumbai|
|17466|Customer_17466|   Mumbai|
|17467|Customer_17467|Bangalore|
|17468|Customer_17468|Hyderabad|
|17469|Customer_17469|Bangalore|
|17470|Customer_17470|Hyderabad|
|17471|Customer_17471|    Delhi|
|17472|Customer_17472|Hyderabad|
|17473|Customer_17473|Ahmedabad|
|17474|Customer_17474|   Mumbai|
+-----+--------------+---------+
only showing top 20 rows



In [46]:
df2.createOrReplaceTempView('manage_cust')

In [47]:
df2.printSchema()

root
 |-- India: string (nullable = true)
 |-- date: string (nullable = true)
 |-- False: string (nullable = true)



In [48]:
spark.sql('select * from manage_cust limit 5').show()

+-----+--------------+---------+
|India|          date|    False|
+-----+--------------+---------+
|17455|Customer_17455|    Delhi|
|17456|Customer_17456|Bangalore|
|17457|Customer_17457|  Kolkata|
|17458|Customer_17458|Hyderabad|
|17459|Customer_17459|Bangalore|
+-----+--------------+---------+



In [51]:
spark.sql('drop table if exists managed_customers')

DataFrame[]

In [52]:
spark.sql('''
    create table managed_customers 
    as 
    select * from manage_cust
''').show()

25/06/10 04:48:35 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


++
||
++
++



In [53]:
spark.sql('describe extended managed_customers').show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|               India|              string|   NULL|
|                date|              string|   NULL|
|               False|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|             default|       |
|               Table|   managed_customers|       |
|               Owner|                root|       |
|        Created Time|Tue Jun 10 04:48:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 3.5.3|       |
|                Type|             MANAGED|       |
|            Provider|                hive|       |
|    Table Properties|[transient_lastDd...|       |
|          Statistics|          5753 bytes|       |
|           

### External Table

In [21]:
!hdfs dfs -ls /data/

Found 6 items
-rw-r--r--   2 root hadoop      12174 2025-06-09 03:26 /data/customers1.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-03 03:00 /data/customers10.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-05 03:18 /data/customers150.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-04 03:32 /data/customers500.csv
-rw-r--r--   2 root hadoop        511 2025-06-06 02:33 /data/malformed_customers.csv
drwxr-xr-x   - root hadoop          0 2025-06-07 08:08 /data/write_op


In [55]:
spark.sql('drop table external_customers')

DataFrame[]

In [64]:
spark.sql('''
    CREATE EXTERNAL TABLE external_customers2(
         India string,
         date string,
         False string
    )
    USING CSV
    LOCATION '/data/external_data/'
''')

25/06/10 05:03:27 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider CSV. Persisting data source table `spark_catalog`.`default`.`external_customers2` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [57]:
spark.sql('describe extended external_customers').show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                 _c0|              string|   NULL|
|                 _c1|              string|   NULL|
|                 _c2|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|             default|       |
|               Table|  external_customers|       |
|               Owner|                root|       |
|        Created Time|Tue Jun 10 04:54:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 3.5.3|       |
|                Type|            EXTERNAL|       |
|            Provider|                 CSV|       |
|            Location|hdfs://my-first-c...|       |
|       Serde Library|org.apache.hadoop...|       |
|         In

In [61]:
!hdfs dfs -mkdir /data/external_data

In [62]:
!hdfs dfs -cp /data/customers1.csv /data/external_data/

In [63]:
!hdfs dfs -ls /data/external_data/

Found 1 items
-rw-r--r--   2 root hadoop      12174 2025-06-10 05:01 /data/external_data/customers1.csv
